In [1]:
import os

from sleep_events import connect_to_firebase

firebase = connect_to_firebase()

In [2]:
import pandas as pd

docs = firebase.collection('notesExperimental').stream()

note_records = [{'id': doc.id, **doc.to_dict()} for doc in docs]

note_df = pd.DataFrame(note_records)

In [7]:
note_df[note_df['dayAndNightOf'].isnull()]

,id,note,dayAndNightOf,timestampForUTC,timestampWritten,timestampFor,tags,timestampWrittenUTC,timestampUpdated,isPreviousDay


In [34]:
firebase_grouped = note_df.groupby('dayAndNightOf').size()
firebase_grouped

dayAndNightOf
2024-05-03     2
2024-05-04     1
2024-05-05     8
2024-05-06     7
2024-05-07     8
              ..
2025-01-05     6
2025-01-06     8
2025-01-07    10
2025-01-08    14
2025-01-09    11
Length: 250, dtype: int64

In [4]:
note_df[note_df['dayAndNightOf'] == '2024-07-11']

,id,note,dayAndNightOf,timestampForUTC,timestampWritten,timestampFor,tags,timestampWrittenUTC,timestampUpdated,isPreviousDay


In [59]:
cloned_note_records[0]

{'id': '2024-05-03T16_48_08_446_01_00',
 'timestampWritten': 1714751288,
 'note': "Interesting ritalin thing.. It suppresses my inhibition enough to make me respond quicker, and that's really making me realise that I am a bit autistic. Maybe?",
 'tags': ['ritalin', 'inhibition', 'autistic'],
 'timestampFor': 1714751237,
 'dayAndNightOf': '2024-05-03',
 'originalKey': '2024-05-03T16:48:08.446+01:00',
 'dayAndNightOfEpoch': 1714690800}

In [39]:
import meilisearch
import json
import json
import re
import os
import dotenv
dotenv.load_dotenv("../../.env")

client = meilisearch.Client('https://examined-life.co.uk:7700', os.getenv('MEILISEARCH_MASTER_KEY'))
index = client.index('notes')


In [212]:
from meilisearch import Client
import itertools

def get_all_unique_tags(client, index_name):
    """
    Retrieves all unique tags from documents in a Meilisearch index.

    Args:
        client: Meilisearch client instance
        index_name: Name of the index to query

    Returns:
        set: A set of unique tags
    """
    # Get the index
    index = client.index(index_name)

    # Get all documents - we'll paginate to handle large datasets
    limit = 1000
    offset = 0
    all_tags = set()

    while True:
        print(f"Fetching documents from offset {offset}...")
        # Fetch batch of documents
        index = client.index('notes')
        results = index.get_documents({
        'limit': limit,
        'offset': offset
        })
        
        # Break if no more documents
        if not results.results:
            break

        # Extract tags from each document and add to set
        batch_tags = set(itertools.chain.from_iterable(
            doc.tags for doc in results.results
        ))
        all_tags.update(batch_tags)

        # Move to next batch
        offset += limit

    return all_tags

get_all_unique_tags(client, 'notes')

Fetching documents from offset 0...
Fetching documents from offset 1000...
Fetching documents from offset 2000...


{'5htp',
 'adhd',
 'adhd:obsessive-research',
 'adhd:protein',
 'air-purifier',
 'apnea',
 'appetite:exercise',
 'appetite:sun',
 'autistic',
 'awake-early',
 'bath',
 'bedroom-heater',
 'blood-pressure',
 'breathing',
 'chamomile',
 'circadian-rhythm',
 'circadian-rhythm: shower',
 'circadian-rhythm:alarm',
 'circadian-rhythm:daylight-savings',
 'circadian-rhythm:energy',
 'circadian-rhythm:heart-rate:early-evening',
 'circadian-rhythm:lep',
 'circadian-rhythm:lep:late-night',
 'circadian-rhythm:lep:sleepiness',
 'circadian-rhythm:second-wind',
 'circadian-rhythm:summary',
 'cleaning',
 'co2',
 'concerta:27mg',
 'concerta:cleaning',
 'concerta:drinking',
 'concerta:emotional',
 'concerta:ending',
 'concerta:insomnia',
 'concerta:kicking-in',
 'concerta:protein',
 'concerta:settling',
 'concerta:skipped-breakfast',
 'concerta:social',
 'concerta:vitamin-c',
 'context',
 'context:fatigue',
 'data',
 'data:core',
 'data:food',
 'data:lux',
 'data:viatom',
 'data:yasa',
 'dehydration',
 '

In [19]:
from meilisearch import Client
import itertools

def get_all_notes(client, index_name):
    index = client.index(index_name)

    # Get all documents - we'll paginate to handle large datasets
    limit = 1000
    offset = 0
    docs = []

    while True:
        print(f"Fetching documents from offset {offset}...")
        # Fetch batch of documents
        results = index.get_documents({
            'limit': limit,
            'offset': offset
        })

        if not results.results:
            break

        docs.extend([doc for doc in results.results])

        # Move to next batch
        offset += limit

    return docs

all_docs = get_all_notes(client, 'notes')

Fetching documents from offset 0...
Fetching documents from offset 1000...
Fetching documents from offset 2000...
Fetching documents from offset 3000...


In [29]:
dir(all_docs[0])

['_Document__doc',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'dayAndNightOf',
 'dayAndNightOfEpoch',
 'id',
 'note',
 'originalKey',
 'tags',
 'timestampFor',
 'timestampWritten']

In [25]:
[c for c in dir(all_docs[0]) if not c.startswith('_')]

['dayAndNightOf',
 'dayAndNightOfEpoch',
 'id',
 'note',
 'originalKey',
 'tags',
 'timestampFor',
 'timestampWritten']

In [33]:
import pandas as pd

# Extract headers from the first document
headers = [c for c in dir(all_docs[0]) if not c.startswith('_')]

# Convert the list of documents to a DataFrame using the extracted headers
all_docs_dicts = [doc.__dict__ if hasattr(doc, '__dict__') else doc for doc in all_docs]

all_docs_df = pd.DataFrame(all_docs_dicts, columns=headers)

# Display the DataFrame
all_docs_ml_grouped = all_docs_df.groupby('dayAndNightOf').size()
all_docs_ml_grouped

dayAndNightOf
2024-05-03     2
2024-05-04     1
2024-05-05     8
2024-05-06     7
2024-05-07     8
              ..
2025-01-05     6
2025-01-06     8
2025-01-07    10
2025-01-08    14
2025-01-09    11
Length: 250, dtype: int64

In [37]:
firebase_grouped

dayAndNightOf
2024-05-03     2
2024-05-04     1
2024-05-05     8
2024-05-06     7
2024-05-07     8
              ..
2025-01-05     6
2025-01-06     8
2025-01-07    10
2025-01-08    14
2025-01-09    11
Length: 250, dtype: int64

In [36]:
comparison = firebase_grouped.compare(all_docs_ml_grouped)
comparison

,self,other
dayAndNightOf,,


In [42]:
all_docs_df[all_docs_df['dayAndNightOf'] == '2024-07-12']

,dayAndNightOf,dayAndNightOfEpoch,id,note,originalKey,tags,timestampFor,timestampWritten
309,2024-07-12,1720738800,2024-07-12T10_21_13_254_01_00,Am feeling a lot better this morning.\nGot the...,2024-07-12T10:21:13.254+01:00,"[sleep:summary, sleep:temp]",1.720776e+09,1.720776e+09
310,2024-07-12,1720738800,2024-07-12T14_36_12_049_01_00,Today has been pretty good until now when I'm ...,2024-07-12T14:36:12.049+01:00,"[co2, sleep:summary, energy:good, sleep:good, ...",1.720791e+09,1.720791e+09
311,2024-07-12,1720738800,2024-07-13T13_15_33_200_01_00,Night of 12th: Co2 change: pumped air out alon...,2024-07-13T13:15:33.200+01:00,[co2],NaN,1.720971e+09


In [50]:
index.search("", {
    "filter": "dayAndNightOfEpoch >= 1720742400  AND dayAndNightOfEpoch <= 1720742400 ",
    "limit": 1000,
    "offset": 0
})

{'hits': [{'note': 'Am feeling a lot better this morning.\nGot the bedroom nice and cool yday.',
   'timestampFor': 1720776073,
   'timestampWritten': 1720776073,
   'dayAndNightOf': '2024-07-12',
   'tags': ['sleep:summary', 'sleep:temp'],
   'id': '2024-07-12T10_21_13_254_01_00',
   'originalKey': '2024-07-12T10:21:13.254+01:00',
   'dayAndNightOfEpoch': 1720742400},
  {'note': "Today has been pretty good until now when I'm slumping after sandwich lunch.  Yday:\nDinner was brown rice + chicken and veg, 7 to 8 ish.\nWorked on XL until 9.  Watched Clarksons Farm, upstairs about 10.  Had a shower.  Was in bed at 10.45 after sorting out EEG montage, fans, windows etc.  Was pretty wired.  \nListened to podcast and used Dodow to settle, same as previous day.  Worked ok, less well.  Charlie still wasn't upstairs and I think I was lights out about 11.30 and she finally came up a bit later (unusual).  \nDifferences from usual: only 2nd night of using podcast.  Haven't had wholegrain carbs in 

In [ ]:
# Query
# 1720742400  == Friday, 12 July 2024 00:00:00, Friday, 12 July 2024 01:00:00 GMT+01:00 DST
# Documents
# 1720738800 == Thursday, 11 July 2024 23:00:00 UTC, Friday, 12 July 2024 00:00:00 GMT+01:00 DST

In [48]:
from datetime import datetime, timezone

dt = datetime.strptime("2024-07-12", '%Y-%m-%d').replace(tzinfo=timezone.utc)
int(dt.timestamp())


1720742400